In [37]:
from ultralytics import YOLO
import cv2
import time
import torch
import pygame  # Import pygame for sound playback

In [39]:
# Checking if CUDA is available
print(torch.cuda.is_available())  # Should return True if CUDA is available

# Get the current device
print(torch.cuda.current_device())  # Returns the current GPU device ID

# Getting the name of the GPU
print(torch.cuda.get_device_name(0))  

True
0
NVIDIA GeForce RTX 2070 Super


## Yolov11 Custom trained Drowsiness Detection Model

In [28]:
# Loading the custom YOLOv11 model
model = YOLO("C:/Users/MAM/Downloads/best (9).pt")

In [35]:
# Check if CUDA is available and transfer the model to GPU
if torch.cuda.is_available():
    device = torch.device('cuda')  # Use GPU
    model.to(device)  # Transfer the model to the GPU
    print("Model is running on GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')  # Use CPU
    print("CUDA not available. Running on CPU.")

Model is running on GPU: NVIDIA GeForce RTX 2070 Super


## Real-Time Detection

In [ ]:
import time
import cv2
import pygame  # Import pygame for sound playback

# Initialize pygame for sound playback
pygame.mixer.init()

# Initialize variables to track drowsiness detection time
drowsy_start_time = None
drowsy_duration = 0
alarm_duration_threshold = 3  # Set the threshold to 3 seconds

# Path to the downloaded alarm sound file
alarm_sound_path = r"C:\Users\MAM\Downloads\714404__vyclops__burglar-alarm-siren-long-alert-warning-sound.mp3"

# Load the alarm sound into pygame's mixer
alarm_sound = pygame.mixer.Sound(alarm_sound_path)

# Variables for blinking "WAKE UP!"
text_visible = True  # Track if the text is currently visible
text_blink_interval = 0.3  # Interval for blinking (0.5 seconds)
last_blink_time = time.time()  # Time when the text was last toggled

# Start the video feed from the webcam
results = model(source=0, stream=True, conf=0.5)

# Process each frame in the video feed
for result in results:
    frame = result.orig_img  # Get the original frame (image)

    # Get class names from the model
    class_names = model.names  # List of class names (e.g., ["alert", "drowsy"])

    drowsy_detected = False  # Flag to check if drowsy is detected in the current frame

    # Loop through each detected box in the current frame
    for box in result.boxes:
        # Get class ID and confidence score
        class_id = int(box.cls[0])
        confidence = box.conf[0]

        # Get the label from class ID
        label = class_names[class_id]

        # Get bounding box coordinates
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Set bounding box color: red for "drowsy", green for others
        if label.lower() == "drowsy":
            color = (0, 0, 255)  # Red (BGR format in OpenCV)
            drowsy_detected = True
        else:
            color = (0, 255, 0)  # Green (BGR format in OpenCV)

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Check if drowsiness is detected across frames
    if drowsy_detected:
        if drowsy_start_time is None:
            drowsy_start_time = time.time()  # Start tracking time when drowsiness is first detected
        else:
            drowsy_duration = time.time() - drowsy_start_time  # Calculate how long drowsiness has been detected

        # If drowsiness is detected for more than 3 seconds, trigger the alarm
        if drowsy_duration >= alarm_duration_threshold:
            print("ALARM! Drowsiness detected for more than 3 seconds.")
            if not pygame.mixer.get_busy():  # Check if sound is already playing
                alarm_sound.play()  # Play the alarm sound asynchronously

            # Blinking "WAKE UP!" message logic
            current_time = time.time()
            if current_time - last_blink_time >= text_blink_interval:
                text_visible = not text_visible  # Toggle text visibility
                last_blink_time = current_time  # Reset blink timer

            # Show "WAKE UP!" message if text is visible
            if text_visible:
                cv2.putText(frame, "WAKE UP!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)
    else:
        # Reset if drowsiness is not detected and stop the alarm
        if pygame.mixer.get_busy():
            alarm_sound.stop()  # Stop the alarm if it's currently playing

        drowsy_start_time = None
        drowsy_duration = 0
        text_visible = True  # Reset text visibility for next time

    # Show the frame with bounding boxes and labels
    cv2.imshow("Drowsiness Detection", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release any OpenCV windowsq
cv2.destroyAllWindows()